In [14]:
# in order to allow the comparison, the filter from the other notebook needs to run and the predictions, too
%run author_vision.ipynb
conversation_ids = set(df.conversation_id.tolist())
df_baseline = df_baseline[df_baseline["conversation_id"].isin(conversation_ids)]


cuda gpu is available: True


2022-08-03 14:59:30.596970: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 14:59:30.597301: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 14:59:30.597527: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 14:59:30.597807: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-03 14:59:30.598025: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

chosen 546 conversations and gotten 535271 from twitter compared to 514793 from reddit
reddit:
0    453046
1     61747
Name: y, dtype: int64
twitter:
0    510363
1     24908
Name: y, dtype: int64
(428216, 81)
(428216,)
2.6.0
3346/3346 [==============================] - 3s 884us/step - loss: 0.0983 - accuracy: 0.9531 - mae: 0.0626
the accuracy on the training set is 0.9530801773071289 and the mae is 0.06255020946264267
(411834, 81)
(411834,)
2.6.0
3218/3218 [==============================] - 3s 851us/step - loss: 0.3278 - accuracy: 0.8807 - mae: 0.1930
the accuracy on the training set is 0.8807486295700073 and the mae is 0.19301630556583405
         timedelta  root_distance_0  reply_distance_2  reply_distance_3  \
1805165   0.000020                1               1.0               0.0   
2442727   0.000166                0               1.0               0.0   

         reply_distance_4  root_distance_1  root_distance_2  root_distance_3  \
1805165               0.0              0.0    

2022-08-03 15:01:11.965255: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 333585864 exceeds 10% of free system memory.


In [15]:
# sum all reply columns

reply_filter_col = [col for col in df_baseline if col.startswith('reply')]
root_distance_filter_col = [col for col in df_baseline if col.startswith('root')]
reply_filter_col

['reply_distance_1',
 'reply_distance_2',
 'reply_distance_3',
 'reply_distance_4',
 'reply_distance_5',
 'reply_distance_6',
 'reply_distance_7',
 'reply_distance_8',
 'reply_distance_9',
 'reply_distance_10',
 'reply_distance_11',
 'reply_distance_12',
 'reply_distance_13',
 'reply_distance_14',
 'reply_distance_15',
 'reply_distance_16',
 'reply_distance_17',
 'reply_distance_18',
 'reply_distance_19',
 'reply_distance_20',
 'reply_distance_21',
 'reply_distance_22',
 'reply_distance_23']

In [16]:
reply_columns = df_baseline[reply_filter_col]
root_distance_columns= df_baseline[root_distance_filter_col]
reply_columns

,reply_distance_1,reply_distance_2,reply_distance_3,reply_distance_4,reply_distance_5,reply_distance_6,reply_distance_7,reply_distance_8,reply_distance_9,reply_distance_10,...,reply_distance_14,reply_distance_15,reply_distance_16,reply_distance_17,reply_distance_18,reply_distance_19,reply_distance_20,reply_distance_21,reply_distance_22,reply_distance_23
6928,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6929,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6931,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6932,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
809819,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
809820,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
809821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
reply_cs = reply_columns.sum(axis=1)
root_distance_cs = root_distance_columns.sum(axis=1)

rcs_not_null = [i for i in reply_cs.tolist() if i != 0]

In [18]:
root_reply_combined = (root_distance_cs + reply_cs)
root_reply_combined = (root_reply_combined - root_reply_combined.min()) / (root_reply_combined.max() - root_reply_combined.min())
combined = [i for i in root_reply_combined.tolist() if i != 0]

In [19]:
#
df_baseline = df_baseline.assign(root_reply_combined=root_reply_combined)
df_baseline

,timedelta,root_distance_0,current,beam_node,has_followed_path,has_follow_path,platform,conversation_id,reply_distance_1,reply_distance_2,...,root_distance_15,root_distance_16,root_distance_17,root_distance_18,root_distance_19,root_distance_20,root_distance_21,root_distance_22,root_distance_23,root_reply_combined
6928,10640.0,1,1543928098890125314,1543883472695185408,0,0,twitter,1543883472695185408,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6929,10697.0,1,1543928341153071104,1543883472695185408,0,0,twitter,1543883472695185408,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6930,57.0,0,1543928341153071104,1543928098890125314,0,0,twitter,1543883472695185408,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
6931,10697.0,1,1543928339173343236,1543883472695185408,0,0,twitter,1543883472695185408,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6932,57.0,0,1543928339173343236,1543928098890125314,0,0,twitter,1543883472695185408,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
809818,47015.0,0,1542589964856070144,1542392772342300672,0,0,twitter,1542362381661024256,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
809819,39754.0,0,1542589964856070144,1542423225153486851,0,0,twitter,1542362381661024256,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
809820,34536.0,0,1542589964856070144,1542445109698039808,0,0,twitter,1542362381661024256,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
809821,8943.0,0,1542589964856070144,1542552458227884033,0,0,twitter,1542362381661024256,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5


In [30]:
import numpy as np

df_data = df_baseline[["root_reply_combined", "conversation_id", "current", "platform"]]
# df_reshaped = pd.pivot_table(df_data,index=["conversation_id", "current"], columns=["root_reply_combined"],aggfunc = np.mean)
baseline_gpm = df_data.groupby(["platform", "conversation_id",  "current"]).mean()
baseline_gpm

root_reply_combined
platform conversation_id     current                                 
reddit   661614              26871177                        0.500000
                             29352234                        0.100000
                             33390443                        0.083333
                             36457165                        0.166667
                             91649333                        0.125000
...                                                               ...
twitter  1544052331481890825 1544069454732046337             0.055556
                             1544069830692818951             0.050000
                             1544083902448410627             0.045455
                             1544084722455805953             0.041667
                             1544097279497437185             0.038462

[4369 rows x 1 columns]

In [21]:
# gpm = gpm.drop(["delab"])
# gpm

In [31]:
baseline_gpm_conversation = baseline_gpm.groupby(by=["platform", "conversation_id"]).mean()
baseline_gpm_conversation

root_reply_combined
platform conversation_id                         
reddit   661614                          0.204167
         10955776                        0.531091
         15848916                        0.228333
         17286490                        0.532382
         19851732                        0.606780
...                                           ...
twitter  1543960369722425345             0.122313
         1543973588545507329             0.347090
         1543990285700632578             0.541667
         1544011512146272257             0.137267
         1544052331481890825             0.122313

[121 rows x 1 columns]

In [32]:
baseline_gpm = baseline_gpm_conversation.groupby(by=["platform"]).mean()
baseline_gpm

,root_reply_combined
platform,
reddit,0.518163
twitter,0.466651


In [39]:
cross_method_gpm = baseline_gpm_conversation.join(gpm_per_conversation, how='inner')
# cross_method_gpm
cross_method_gpm.corr()

,root_reply_combined,predictions
root_reply_combined,1.000000,0.074859
predictions,0.074859,1.000000
